In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore
import scipy
import seaborn
import pickle
from sklearn import decomposition, manifold
from rastermap import Rastermap, utils
import pandas as pd

In [ ]:
def generate_continuous_data(data, event, trial, padding,delay,interval,percentile,gaussian):
    data = data.T

    if interval > 0:
        for i in range(int(data.shape[0]/interval)+1):
            baseline = np.mean(data[i*interval:(i+1)*interval,:])
            data[i*interval:(i+1)*interval,:] = data[i*interval:(i+1)*interval,:]/baseline
    
    for n in range(data.shape[1]):
        threshold = np.percentile(data[:,n][data[:,n]>0],percentile)
        data[:,n][data[:,n]<threshold] = 0
        
    data = scipy.ndimage.gaussian_filter1d(data,2,axis=0)

    data = data.T
    
    data = scipy.stats.zscore(data, axis=1)
    data = data[:, round(event[trial[0]])-padding[0]:round(event[trial[1]])+padding[1]] 
    
    event_index = []
    for i in range(trial[0], trial[1]+1):
        event_index.append(round(event[i])-(round(event[trial[0]])-padding[0]))
    
    data = np.array(data)
    event_index = np.array(event_index)

    # Substract delay
    delay = delay
    event_index_new = []
    for i,e in enumerate(event_index):
        event_index_new.append(round(e+i*delay))
    event_index = event_index_new

    return data, event_index

In [ ]:
def PCA(data, event_index, n_components, label=[],mask_period=[-20,50],):
    
    reducer = decomposition.PCA(n_components)
    mask = np.ones([data.shape[1]],dtype=bool)
    for i in event_index:
        mask[i+mask_period[0]:i+mask_period[1]] = False
    
    # Transpose
    data = data.T
    
    # Fit model
    reducer.fit(data[mask])

    # Transform
    embeddings = reducer.transform(data)
    print(reducer.explained_variance_ratio_)

    if label == []:
        for i in range(n_components):
            plt.figure(figsize=(150, 5), dpi=80)
            plt.plot(np.array(range(len(embeddings[:,i])))/10,-embeddings[:,i])
            
            for j in event_index:
                plt.plot([j/10-2,j/10-2], [-5, 5], color='grey', linestyle='--')
            plt.ylim(-15,15)
            plt.show()
    else:
        for pc in range(n_components):
            success_trials = []
            failure_trials = []
            nogo_trials = []
            miss_trials = []
            
            plt.figure(figsize=(150, 5), dpi=80)
            plt.plot(np.array(range(len(embeddings[:,pc])))/10,-embeddings[:,pc])
                
            for i,j in enumerate(event_index):
                plt.plot([j/10+5,j/10+5], [-5, 5], color='grey', linestyle='--')
                if label[i] == "ss" or label[i] =="ms":
                    plt.plot([j/10-2,j/10-2], [-5, 5], color='green', linestyle='--')
                    success_trials.append(embeddings[j-50:j+200,pc])
                elif label[i] == "sf" or label[i] =="mf":
                    plt.plot([j/10-2,j/10-2], [-5, 5], color='red', linestyle='--')
                    failure_trials.append(embeddings[j-50:j+200,pc])
                elif label[i] == "n":
                    plt.plot([j/10-2,j/10-2], [-5, 5], color='black', linestyle='--')
                    nogo_trials.append(embeddings[j-50:j+200,pc])
                else:
                    plt.plot([j/10-2,j/10-2], [-5, 5], color='blue', linestyle='--')
                    miss_trials.append(embeddings[j-50:j+200,pc])

            plt.ylim(-15,15)
            plt.show()

            plt.plot(np.mean(success_trials,axis=0),color='green')
            plt.plot(np.mean(failure_trials,axis=0),color='red')
            plt.plot(np.mean(nogo_trials,axis=0),color='black')
            # plt.plot(np.mean(miss_trials,axis=0),color='blue')
            plt.show()

# plt.show()

    # plt.plot(embeddings[:,0],embeddings[:,1])
    # plt.show()

    # plt.plot(embeddings[:,0],embeddings[:,2])
    # plt.show()

    # plt.plot(embeddings[:,1],embeddings[:,2])
    # plt.show()
        
    # sorted_order = np.argsort(np.arctan(reducer.components_[1]/reducer.components_[0]))
    # fig = plt.figure(figsize=(150,2))
    # seaborn.heatmap(data.T[sorted_order],vmin=0, vmax=2,cmap="gray_r")
    # plt.show()



In [ ]:
# Old data
path = 'E:/CK-87-data/2023-04-05/'
filename = path+"2023-04-05_cnmf.hdf5"
Trial_idx_file = path+'Trial_idx1.mat'
Trial_idx = scipy.io.loadmat(Trial_idx_file)['Trial_idx1']
door_open = Trial_idx[:,1]
with h5py.File(filename, "r") as f:
    data = f['estimates']['S_dff'][:]
print(data.shape)

data, door_open_index = generate_continuous_data(data, door_open, [0,len(door_open)-1], [100,200],0.1,1000,30,0.1)
PCA(data, door_open_index, 3, [0,50])

In [ ]:
# Old data
path = 'E:/CK-92-Processed/2024-01-16/'
filename = path+"2024-01-16_cnmf.hdf5"
Trial_idx_file = path+'Trial_idx1.mat'
Trial_idx = scipy.io.loadmat(Trial_idx_file)['Trial_idx1']
door_open = Trial_idx[:,1]
with h5py.File(filename, "r") as f:
    data = f['estimates']['S_dff'][:]
print(data.shape)

data, door_open_index = generate_continuous_data(data, door_open, [0,len(door_open)-1], [100,200],0.1,1000,30,0.1)
PCA(data, door_open_index, 3, [0,50])

In [ ]:
# Old data
path = 'E:/LM-26-CaMK-GCaMP6f/2024-04-26/'
filename = path+"2024-04-26_cnmf.hdf5"
Trial_idx_file = path+'Trial_idx1.mat'
Trial_idx = scipy.io.loadmat(Trial_idx_file)['Trial_idx1']
door_open = Trial_idx[:,1]
with h5py.File(filename, "r") as f:
    data = f['estimates']['S_dff'][:]
print(data.shape)

data, door_open_index = generate_continuous_data(data, door_open, [0,len(door_open)-1], [100,200],0.1,1000,30,0.1)
PCA(data, door_open_index, 3, [0,50])

In [ ]:
# Old data
path = 'E:/LM-27-CaMK-GCaMP6f/2024-04-26/'
filename = path+"2024-04-26_cnmf.hdf5"
Trial_idx_file = path+'Trial_idx1.mat'
Trial_idx = scipy.io.loadmat(Trial_idx_file)['Trial_idx1']
door_open = Trial_idx[:,1]
with h5py.File(filename, "r") as f:
    data = f['estimates']['S_dff'][:]
print(data.shape)

data, door_open_index = generate_continuous_data(data, door_open, [0,len(door_open)-1], [100,200],0.1,1000,30,0.1)
PCA(data, door_open_index, 3, [0,50])

In [ ]:
# Old data
path = 'E:/LM-15-Processed-SNR4PNR14/2021-12-29/'
filename = path+"2021-12-29_cnmf.hdf5"
Trial_idx_file = path+'Trial_idx1.mat'
Trial_idx = scipy.io.loadmat(Trial_idx_file)['Trial_idx1']
door_open = Trial_idx[:,1]
with h5py.File(filename, "r") as f:
    data = f['estimates']['S_dff'][:]
print(data.shape)

data, door_open_index = generate_continuous_data(data, door_open, [0,len(door_open)-1], [100,200],0.1,1000,30,0.1)
PCA(data, door_open_index, 3, [0,50])

In [ ]:
# Old data
path = 'E:/LM-11-Processed/2021-12-27/'
filename = path+"2021-12-27_cnmf.hdf5"
Trial_idx_file = path+'Trial_idx1.mat'
Trial_idx = scipy.io.loadmat(Trial_idx_file)['Trial_idx1']
door_open = Trial_idx[:,1]
with h5py.File(filename, "r") as f:
    data = f['estimates']['S_dff'][:]
print(data.shape)

data, door_open_index = generate_continuous_data(data, door_open, [0,len(door_open)-1], [100,200],0.1,1000,30,0.1)
PCA(data, door_open_index, 3, [0,50])

In [ ]:
path = 'G:/mPFC-2/2025-05-07/'
filename = path+"2025-05-07_cnmf.hdf5"
Trial_idx_file = path+'Trial_idx1.mat'
Trial_idx = scipy.io.loadmat(Trial_idx_file)['Trial_idx1']
door_open = Trial_idx[:,1]
with h5py.File(filename, "r") as f:
    data = f['estimates']['S_dff'][:]
print(data.shape)
data, door_open_index = generate_continuous_data(data, door_open, [0,len(door_open)-1], [100,200],0.13,1000,50,0.1)
print(data.shape)
# Load behavior file
behav_file = path+'behav_score.xlsx'
bf = pd.read_excel(behav_file, header=None)
label = np.squeeze(bf.values)

print(label)
PCA(data, door_open_index, 3)

# Rastermap
model = Rastermap(locality=0.05, time_lag_window=20).fit(data)
sorted_order = model.isort
fig = plt.figure(figsize=(150,2))
seaborn.heatmap(data[sorted_order],vmin=0,vmax=5,cmap="gray_r")
for i,e in enumerate(door_open_index):
    if label[i] == "ss" or label[i] =="ms":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='green',linestyle='--', linewidth=0.4)
    elif label[i] == "sf" or label[i] =="mf":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='red',linestyle='--', linewidth=0.4)
    elif label[i] == "n":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='black',linestyle='--', linewidth=0.4)
    else:
        plt.plot([e-20,e-20],[0,data.shape[0]], color='blue',linestyle='--', linewidth=0.4)
plt.show()

In [ ]:
path = 'G:/M2-5/2025-04-06/'
filename = path+"2025-04-06_cnmf.hdf5"
Trial_idx_file = path+'Trial_idx1.mat'
Trial_idx = scipy.io.loadmat(Trial_idx_file)['Trial_idx1']
door_open = Trial_idx[:,1]
with h5py.File(filename, "r") as f:
    data = f['estimates']['S_dff'][:]
data, door_open_index = generate_continuous_data(data, door_open, [0,len(door_open)-1], [100,1000],0.13,3000,30,0.1)


# Load behavior file
behav_file = path+'behav_score.xlsx'
bf = pd.read_excel(behav_file, header=None)
label = np.squeeze(bf.values)

PCA(data, door_open_index,5,label)



print(label)
print(data.shape)

# # Rastermap
# model = Rastermap(locality=0.5, time_lag_window=20).fit(data)
# sorted_order = model.isort


# fig = plt.figure(figsize=(150,2))
# seaborn.heatmap(data[sorted_order],vmin=0,vmax=5,cmap="gray_r")


# success_data = []
# failure_data = []
# for i,e in enumerate(door_open_index):
#     if label[i] == "ss" or label[i] =="ms":
#         plt.plot([e-20,e-20],[0,data.shape[0]], color='green',linestyle='--', linewidth=0.4)
#         success_data.append(data[sorted_order,e-20:e+100])
#     elif label[i] == "sf" or label[i] =="mf":
#         plt.plot([e-20,e-20],[0,data.shape[0]], color='red',linestyle='--', linewidth=0.4)
#         failure_data.append(data[sorted_order,e-20:e+100])
#     elif label[i] == "n":
#         plt.plot([e-20,e-20],[0,data.shape[0]], color='black',linestyle='--', linewidth=0.4)
#     else:
#         plt.plot([e-20,e-20],[0,data.shape[0]], color='blue',linestyle='--', linewidth=0.4)
# plt.show()

# for i,e in enumerate(door_open_index):
#     plt.plot([e-20,e-20],[0,data.shape[0]], color='blue',linestyle='--', linewidth=0.4)
# plt.show()

# success_data = np.average(success_data,axis=0)
# seaborn.heatmap(success_data)
# plt.show()
# failure_data_1 = np.average(failure_data[:10],axis=0)
# seaborn.heatmap(failure_data_1,vmin=0,vmax=4)
# plt.show()
# failure_data_2 = np.average(failure_data[-10:],axis=0)
# seaborn.heatmap(failure_data_2,vmin=0,vmax=4)
# plt.show()

In [ ]:
path = 'G:/mPFC-1/2025-05-07/'
filename = path+"2025-05-07_cnmf.hdf5"
Trial_idx_file = path+'Trial_idx1.mat'
Trial_idx = scipy.io.loadmat(Trial_idx_file)['Trial_idx1']
door_open = Trial_idx[:,1]
with h5py.File(filename, "r") as f:
    data = f['estimates']['S_dff'][:]
print(data.shape)
data, door_open_index = generate_continuous_data(data, door_open, [0,len(door_open)-1], [100,200],0.14,1000,30,0.1)
PCA(data, door_open_index, 3)

In [ ]:
path = 'G:/mPFC-1/2025-05-29/'
filename = path+"2025-05-29_cnmf.hdf5"
Trial_idx_file = path+'Trial_idx1.mat'
Trial_idx = scipy.io.loadmat(Trial_idx_file)['Trial_idx1']
door_open = Trial_idx[:,1]
with h5py.File(filename, "r") as f:
    data = f['estimates']['S_dff'][:]
print(data.shape)
data, door_open_index = generate_continuous_data(data, door_open, [0,len(door_open)-1], [100,200],0.13,1000,30,0.1)
PCA(data, door_open_index, 3)

# Load behavior file
behav_file = path+'behav_score.xlsx'
bf = pd.read_excel(behav_file, header=None)
label = np.squeeze(bf.values)

print(label)
print(data.shape)

# Rastermap
model = Rastermap(locality=0.5, time_lag_window=20).fit(data)
sorted_order = model.isort


fig = plt.figure(figsize=(150,2))
seaborn.heatmap(data[sorted_order],vmin=0,vmax=5,cmap="gray_r")
success_data = []
failure_data = []
for i,e in enumerate(door_open_index):
    if label[i] == "ss" or label[i] =="ms":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='green',linestyle='--', linewidth=0.4)
        success_data.append(data[sorted_order,e-20:e+100])
    elif label[i] == "sf" or label[i] =="mf":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='red',linestyle='--', linewidth=0.4)
        failure_data.append(data[sorted_order,e-20:e+100])
    elif label[i] == "n":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='black',linestyle='--', linewidth=0.4)
    else:
        plt.plot([e-20,e-20],[0,data.shape[0]], color='blue',linestyle='--', linewidth=0.4)
plt.show()

success_data = np.average(success_data,axis=0)
seaborn.heatmap(success_data)
plt.show()
failure_data_1 = np.average(failure_data[:10],axis=0)
seaborn.heatmap(failure_data_1,vmin=0,vmax=7)
plt.show()
failure_data_2 = np.average(failure_data[-10:],axis=0)
seaborn.heatmap(failure_data_2,vmin=0,vmax=7)
plt.show()

In [ ]:
path = 'G:/mPFC-3/2025-07-05/'
filename = path+"2025-07-05_cnmf.hdf5"
Trial_idx_file = path+'Trial_idx1.mat'
Trial_idx = scipy.io.loadmat(Trial_idx_file)['Trial_idx1']
door_open = Trial_idx[:,1]
with h5py.File(filename, "r") as f:
    data = f['estimates']['S_dff'][:]
print(data.shape)

data, door_open_index = generate_continuous_data(data, door_open, [0,len(door_open)-1], [100,200],0.13,1000,30,0.1)
PCA(data, door_open_index, 3)

# Load behavior file
behav_file = path+'behav_score.xlsx'
bf = pd.read_excel(behav_file, header=None)
label = np.squeeze(bf.values)

print(label)
print(data.shape)

# Rastermap
model = Rastermap(locality=0.5, time_lag_window=20).fit(data)
sorted_order = model.isort


fig = plt.figure(figsize=(150,2))
seaborn.heatmap(data[sorted_order],vmin=0,vmax=5,cmap="gray_r")
success_data = []
failure_data = []
for i,e in enumerate(door_open_index):
    if label[i] == "ss" or label[i] =="ms":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='green',linestyle='--', linewidth=0.4)
        success_data.append(data[sorted_order,e-20:e+100])
    elif label[i] == "sf" or label[i] =="mf":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='red',linestyle='--', linewidth=0.4)
        failure_data.append(data[sorted_order,e-20:e+100])
    elif label[i] == "n":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='black',linestyle='--', linewidth=0.4)
    else:
        plt.plot([e-20,e-20],[0,data.shape[0]], color='blue',linestyle='--', linewidth=0.4)
plt.show()

success_data = np.average(success_data,axis=0)
seaborn.heatmap(success_data)
plt.show()
failure_data_1 = np.average(failure_data[:10],axis=0)
seaborn.heatmap(failure_data_1,vmin=0,vmax=7)
plt.show()
failure_data_2 = np.average(failure_data[-10:],axis=0)
seaborn.heatmap(failure_data_2,vmin=0,vmax=7)
plt.show()

In [ ]:
path = 'G:/mPFC-3/2025-07-12/'
filename = path+"2025-07-12_cnmf.hdf5"
Trial_idx_file = path+'Trial_idx1.mat'
Trial_idx = scipy.io.loadmat(Trial_idx_file)['Trial_idx1']
door_open = Trial_idx[:,1]
with h5py.File(filename, "r") as f:
    data = f['estimates']['S_dff'][:]
print(data.shape)
data, door_open_index = generate_continuous_data(data, door_open, [0,len(door_open)-1], [100,200],0.13,1000,30,0.1)
PCA(data, door_open_index, 3)

# Load behavior file
behav_file = path+'behav_score.xlsx'
bf = pd.read_excel(behav_file, header=None)
label = np.squeeze(bf.values)

print(label)
print(data.shape)

# Rastermap
model = Rastermap(locality=0.5, time_lag_window=20).fit(data)
sorted_order = model.isort


fig = plt.figure(figsize=(150,2))
seaborn.heatmap(data[sorted_order],vmin=0,vmax=5,cmap="gray_r")
success_data = []
failure_data = []
for i,e in enumerate(door_open_index):
    if label[i] == "ss" or label[i] =="ms":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='green',linestyle='--', linewidth=0.4)
        success_data.append(data[sorted_order,e-20:e+100])
    elif label[i] == "sf" or label[i] =="mf":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='red',linestyle='--', linewidth=0.4)
        failure_data.append(data[sorted_order,e-20:e+100])
    elif label[i] == "n":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='black',linestyle='--', linewidth=0.4)
    else:
        plt.plot([e-20,e-20],[0,data.shape[0]], color='blue',linestyle='--', linewidth=0.4)
plt.show()

success_data = np.average(success_data,axis=0)
seaborn.heatmap(success_data)
plt.show()
failure_data_1 = np.average(failure_data[:10],axis=0)
seaborn.heatmap(failure_data_1,vmin=0,vmax=7)
plt.show()
failure_data_2 = np.average(failure_data[-10:],axis=0)
seaborn.heatmap(failure_data_2,vmin=0,vmax=7)
plt.show()

In [ ]:
path = 'G:/mPFC-5/2025-07-05/'
filename = path+"2025-07-05_cnmf.hdf5"
Trial_idx_file = path+'Trial_idx1.mat'
Trial_idx = scipy.io.loadmat(Trial_idx_file)['Trial_idx1']
door_open = Trial_idx[:,1]
with h5py.File(filename, "r") as f:
    data = f['estimates']['S_dff'][:]
print(data.shape)


data, door_open_index = generate_continuous_data(data, door_open, [0,len(door_open)-1], [100,200],0.13,1000,30,0.1)
PCA(data, door_open_index, 3)

# Load behavior file
behav_file = path+'behav_score.xlsx'
bf = pd.read_excel(behav_file, header=None)
label = np.squeeze(bf.values)

print(label)
print(data.shape)

# Rastermap
model = Rastermap(locality=0.5, time_lag_window=20).fit(data)
sorted_order = model.isort


fig = plt.figure(figsize=(150,2))
seaborn.heatmap(data[sorted_order],vmin=0,vmax=5,cmap="gray_r")
success_data = []
failure_data = []
for i,e in enumerate(door_open_index):
    if label[i] == "ss" or label[i] =="ms":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='green',linestyle='--', linewidth=0.4)
        success_data.append(data[sorted_order,e-20:e+100])
    elif label[i] == "sf" or label[i] =="mf":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='red',linestyle='--', linewidth=0.4)
        failure_data.append(data[sorted_order,e-20:e+100])
    elif label[i] == "n":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='black',linestyle='--', linewidth=0.4)
    else:
        plt.plot([e-20,e-20],[0,data.shape[0]], color='blue',linestyle='--', linewidth=0.4)
plt.show()

success_data = np.average(success_data,axis=0)
seaborn.heatmap(success_data)
plt.show()
failure_data_1 = np.average(failure_data[:10],axis=0)
seaborn.heatmap(failure_data_1,vmin=0,vmax=7)
plt.show()
failure_data_2 = np.average(failure_data[-10:],axis=0)
seaborn.heatmap(failure_data_2,vmin=0,vmax=7)
plt.show()

In [ ]:
path = 'G:/mPFC-5/2025-07-14/'
filename = path+"2025-07-14_cnmf.hdf5"
Trial_idx_file = path+'Trial_idx1.mat'
Trial_idx = scipy.io.loadmat(Trial_idx_file)['Trial_idx1']
door_open = Trial_idx[:,1]
with h5py.File(filename, "r") as f:
    data = f['estimates']['S_dff'][:]
print(data.shape)
data, door_open_index = generate_continuous_data(data, door_open, [0,len(door_open)-1], [100,200],0.13,1000,30,0.1)
PCA(data, door_open_index, 3)

# Load behavior file
behav_file = path+'behav_score.xlsx'
bf = pd.read_excel(behav_file, header=None)
label = np.squeeze(bf.values)

print(label)
print(data.shape)

# Rastermap
model = Rastermap(locality=0.5, time_lag_window=20).fit(data)
sorted_order = model.isort


fig = plt.figure(figsize=(150,2))
seaborn.heatmap(data[sorted_order],vmin=0,vmax=5,cmap="gray_r")
success_data = []
failure_data = []
for i,e in enumerate(door_open_index):
    if label[i] == "ss" or label[i] =="ms":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='green',linestyle='--', linewidth=0.4)
        success_data.append(data[sorted_order,e-20:e+50])
    elif label[i] == "sf" or label[i] =="mf":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='red',linestyle='--', linewidth=0.4)
        failure_data.append(data[sorted_order,e-20:e+50])
    elif label[i] == "n":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='black',linestyle='--', linewidth=0.4)
    else:
        plt.plot([e-20,e-20],[0,data.shape[0]], color='blue',linestyle='--', linewidth=0.4)
plt.show()

success_data = np.average(success_data,axis=0)
seaborn.heatmap(success_data)
plt.show()
failure_data = np.average(failure_data,axis=0)
seaborn.heatmap(failure_data)
plt.show()

In [ ]:
path = 'G:/mPFC-6/2025-07-04/'
filename = path+"2025-07-04_cnmf.hdf5"
Trial_idx_file = path+'Trial_idx1.mat'
Trial_idx = scipy.io.loadmat(Trial_idx_file)['Trial_idx1']
door_open = Trial_idx[:,1]
with h5py.File(filename, "r") as f:
    data = f['estimates']['S_dff'][:]
print(data.shape)



data, door_open_index = generate_continuous_data(data, door_open, [0,len(door_open)-1], [100,200],0.13,1000,30,0.1)
PCA(data, door_open_index, 3)

# Load behavior file
behav_file = path+'behav_score.xlsx'
bf = pd.read_excel(behav_file, header=None)
label = np.squeeze(bf.values)

print(label)
print(data.shape)

# Rastermap
model = Rastermap(locality=0.5, time_lag_window=20).fit(data)
sorted_order = model.isort


fig = plt.figure(figsize=(150,2))
seaborn.heatmap(data[sorted_order],vmin=0,vmax=5,cmap="gray_r")
success_data = []
failure_data = []
for i,e in enumerate(door_open_index):
    if label[i] == "ss" or label[i] =="ms":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='green',linestyle='--', linewidth=0.4)
        success_data.append(data[sorted_order,e-20:e+100])
    elif label[i] == "sf" or label[i] =="mf":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='red',linestyle='--', linewidth=0.4)
        failure_data.append(data[sorted_order,e-20:e+100])
    elif label[i] == "n":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='black',linestyle='--', linewidth=0.4)
    else:
        plt.plot([e-20,e-20],[0,data.shape[0]], color='blue',linestyle='--', linewidth=0.4)
plt.show()

success_data = np.average(success_data,axis=0)
seaborn.heatmap(success_data)
plt.show()
failure_data_1 = np.average(failure_data[:10],axis=0)
seaborn.heatmap(failure_data_1,vmin=0,vmax=7)
plt.show()
failure_data_2 = np.average(failure_data[-10:],axis=0)
seaborn.heatmap(failure_data_2,vmin=0,vmax=7)
plt.show()

In [ ]:
path = 'G:/mPFC-6/2025-07-12/'
filename = path+"2025-07-12_cnmf.hdf5"
Trial_idx_file = path+'Trial_idx1.mat'
Trial_idx = scipy.io.loadmat(Trial_idx_file)['Trial_idx1']
door_open = Trial_idx[:,1]
with h5py.File(filename, "r") as f:
    data = f['estimates']['S_dff'][:]
print(data.shape)
data, door_open_index = generate_continuous_data(data, door_open, [0,len(door_open)-1], [100,200],0.13,1000,30,0.1)
PCA(data, door_open_index, 3)

# Load behavior file
behav_file = path+'behav_score.xlsx'
bf = pd.read_excel(behav_file, header=None)
label = np.squeeze(bf.values)

print(label)
print(data.shape)

# Rastermap
model = Rastermap(locality=0.5, time_lag_window=20).fit(data)
sorted_order = model.isort


fig = plt.figure(figsize=(150,2))
seaborn.heatmap(data[sorted_order],vmin=0,vmax=5,cmap="gray_r")
success_data = []
failure_data = []
for i,e in enumerate(door_open_index):
    if label[i] == "ss" or label[i] =="ms":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='green',linestyle='--', linewidth=0.4)
        success_data.append(data[sorted_order,e-20:e+100])
    elif label[i] == "sf" or label[i] =="mf":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='red',linestyle='--', linewidth=0.4)
        failure_data.append(data[sorted_order,e-20:e+100])
    elif label[i] == "n":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='black',linestyle='--', linewidth=0.4)
    else:
        plt.plot([e-20,e-20],[0,data.shape[0]], color='blue',linestyle='--', linewidth=0.4)
plt.show()

success_data = np.average(success_data,axis=0)
seaborn.heatmap(success_data)
plt.show()
failure_data_1 = np.average(failure_data[:10],axis=0)
seaborn.heatmap(failure_data_1,vmin=0,vmax=7)
plt.show()
failure_data_2 = np.average(failure_data[-10:],axis=0)
seaborn.heatmap(failure_data_2,vmin=0,vmax=7)
plt.show()

In [ ]:
path = 'G:/M2-1/2025-03-26/'
filename = path+"2025-03-26_cnmf.hdf5"
Trial_idx_file = path+'Trial_idx1.mat'
Trial_idx = scipy.io.loadmat(Trial_idx_file)['Trial_idx1']
door_open = Trial_idx[:,1]
with h5py.File(filename, "r") as f:
    data = f['estimates']['S_dff'][:]
print(data.shape)

# Rastermap
model = Rastermap(locality=0.05, time_lag_window=20).fit(data)
sorted_order = model.isort

data, door_open_index = generate_continuous_data(data, door_open, [0,len(door_open)-1], [100,200],0.13,1000,30,0.1)
PCA(data, door_open_index, 3)

# Load behavior file
behav_file = path+'behav_score.xlsx'
bf = pd.read_excel(behav_file, header=None)
label = np.squeeze(bf.values)

print(label)
print(data.shape)



fig = plt.figure(figsize=(150,2))
seaborn.heatmap(data[sorted_order],vmin=0,vmax=5,cmap="gray_r")
success_data = []
failure_data = []
for i,e in enumerate(door_open_index):
    if label[i] == "ss" or label[i] =="ms":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='green',linestyle='--', linewidth=0.4)
        success_data.append(data[sorted_order,e-20:e+50])
    elif label[i] == "sf" or label[i] =="mf":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='red',linestyle='--', linewidth=0.4)
        failure_data.append(data[sorted_order,e-20:e+50])
    elif label[i] == "n":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='black',linestyle='--', linewidth=0.4)
    else:
        plt.plot([e-20,e-20],[0,data.shape[0]], color='blue',linestyle='--', linewidth=0.4)
plt.show()

success_data = np.average(success_data,axis=0)
seaborn.heatmap(success_data)
plt.show()
failure_data = np.average(failure_data,axis=0)
seaborn.heatmap(failure_data)
plt.show()

In [ ]:
path = 'G:/M2-1/2025-04-05/'
filename = path+"2025-04-05_cnmf.hdf5"
Trial_idx_file = path+'Trial_idx1.mat'
Trial_idx = scipy.io.loadmat(Trial_idx_file)['Trial_idx1']
door_open = Trial_idx[:,1]
with h5py.File(filename, "r") as f:
    data = f['estimates']['S_dff'][:]
print(data.shape)
# Rastermap
model = Rastermap(locality=0.05, time_lag_window=20).fit(data)
sorted_order = model.isort

data, door_open_index = generate_continuous_data(data, door_open, [0,len(door_open)-1], [100,200],0.13,1000,30,0.1)
PCA(data, door_open_index, 3)

# Load behavior file
behav_file = path+'behav_score.xlsx'
bf = pd.read_excel(behav_file, header=None)
label = np.squeeze(bf.values)

print(label)
print(data.shape)



fig = plt.figure(figsize=(150,2))
seaborn.heatmap(data[sorted_order],vmin=0,vmax=5,cmap="gray_r")
success_data = []
failure_data = []
for i,e in enumerate(door_open_index):
    if label[i] == "ss" or label[i] =="ms":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='green',linestyle='--', linewidth=0.4)
        success_data.append(data[sorted_order,e-20:e+100])
    elif label[i] == "sf" or label[i] =="mf":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='red',linestyle='--', linewidth=0.4)
        failure_data.append(data[sorted_order,e-20:e+100])
    elif label[i] == "n":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='black',linestyle='--', linewidth=0.4)
    else:
        plt.plot([e-20,e-20],[0,data.shape[0]], color='blue',linestyle='--', linewidth=0.4)
plt.show()

success_data = np.average(success_data,axis=0)
seaborn.heatmap(success_data)
plt.show()
failure_data = np.average(failure_data,axis=0)
seaborn.heatmap(failure_data)
plt.show()

In [ ]:
path = 'G:/M2-2/2025-03-27/'
filename = path+"2025-03-27_cnmf.hdf5"
Trial_idx_file = path+'Trial_idx1.mat'
Trial_idx = scipy.io.loadmat(Trial_idx_file)['Trial_idx1']
door_open = Trial_idx[:,1]
with h5py.File(filename, "r") as f:
    data = f['estimates']['S_dff'][:]
print(data.shape)
data, door_open_index = generate_continuous_data(data, door_open, [0,len(door_open)-1], [100,200],0.14,1000,30,0.1)
PCA(data, door_open_index, 3)

In [ ]:
path = 'G:/M2-2/2025-04-05/'
filename = path+"2025-04-05_cnmf.hdf5"
Trial_idx_file = path+'Trial_idx1.mat'
Trial_idx = scipy.io.loadmat(Trial_idx_file)['Trial_idx1']
door_open = Trial_idx[:,1]
with h5py.File(filename, "r") as f:
    data = f['estimates']['S_dff'][:]
print(data.shape)
data, door_open_index = generate_continuous_data(data, door_open, [0,len(door_open)-1], [100,200],0.14,1000,30,0.1)
PCA(data, door_open_index, 3)

In [ ]:
path = 'G:/M2-3/2025-03-26/'
filename = path+"2025-03-26_cnmf.hdf5"
Trial_idx_file = path+'Trial_idx1.mat'
Trial_idx = scipy.io.loadmat(Trial_idx_file)['Trial_idx1']
door_open = Trial_idx[:,1]
with h5py.File(filename, "r") as f:
    data = f['estimates']['S_dff'][:]
print(data.shape)
# Rastermap
model = Rastermap(locality=0.5, time_lag_window=20).fit(data)
sorted_order = model.isort

data, door_open_index = generate_continuous_data(data, door_open, [0,len(door_open)-1], [100,200],0.13,1000,30,0.1)
PCA(data, door_open_index, 3)

# Load behavior file
behav_file = path+'behav_score.xlsx'
bf = pd.read_excel(behav_file, header=None)
label = np.squeeze(bf.values)

print(label)
print(data.shape)



fig = plt.figure(figsize=(150,2))
seaborn.heatmap(data[sorted_order],vmin=0,vmax=5,cmap="gray_r")
success_data = []
failure_data = []
for i,e in enumerate(door_open_index):
    if label[i] == "ss" or label[i] =="ms":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='green',linestyle='--', linewidth=0.4)
        success_data.append(data[sorted_order,e-20:e+50])
    elif label[i] == "sf" or label[i] =="mf":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='red',linestyle='--', linewidth=0.4)
        failure_data.append(data[sorted_order,e-20:e+50])
    elif label[i] == "n":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='black',linestyle='--', linewidth=0.4)
    else:
        plt.plot([e-20,e-20],[0,data.shape[0]], color='blue',linestyle='--', linewidth=0.4)
plt.show()

success_data = np.average(success_data,axis=0)
seaborn.heatmap(success_data)
plt.show()
failure_data = np.average(failure_data,axis=0)
seaborn.heatmap(failure_data)
plt.show()

In [ ]:
path = 'G:/M2-3/2025-04-05/'
filename = path+"2025-04-05_cnmf.hdf5"
Trial_idx_file = path+'Trial_idx1.mat'
Trial_idx = scipy.io.loadmat(Trial_idx_file)['Trial_idx1']
door_open = Trial_idx[:,1]
with h5py.File(filename, "r") as f:
    data = f['estimates']['S_dff'][:]
print(data.shape)

# Rastermap
model = Rastermap(locality=0.05, time_lag_window=20).fit(data)
sorted_order = model.isort

data, door_open_index = generate_continuous_data(data, door_open, [0,len(door_open)-1], [100,200],0.13,1000,30,0.1)
PCA(data, door_open_index, 3)

# Load behavior file
behav_file = path+'behav_score.xlsx'
bf = pd.read_excel(behav_file, header=None)
label = np.squeeze(bf.values)

print(label)
print(data.shape)



fig = plt.figure(figsize=(150,2))
seaborn.heatmap(data[sorted_order],vmin=0,vmax=5,cmap="gray_r")
success_data = []
failure_data = []
for i,e in enumerate(door_open_index):
    if label[i] == "ss" or label[i] =="ms":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='green',linestyle='--', linewidth=0.4)
        success_data.append(data[sorted_order,e-20:e+50])
    elif label[i] == "sf" or label[i] =="mf":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='red',linestyle='--', linewidth=0.4)
        failure_data.append(data[sorted_order,e-20:e+50])
    elif label[i] == "n":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='black',linestyle='--', linewidth=0.4)
    else:
        plt.plot([e-20,e-20],[0,data.shape[0]], color='blue',linestyle='--', linewidth=0.4)
plt.show()

success_data = np.average(success_data,axis=0)
seaborn.heatmap(success_data)
plt.show()
failure_data = np.average(failure_data,axis=0)
seaborn.heatmap(failure_data)
plt.show()

In [ ]:
path = 'G:/M2-4/2025-03-26/'
filename = path+"2025-03-26_cnmf.hdf5"
Trial_idx_file = path+'Trial_idx1.mat'
Trial_idx = scipy.io.loadmat(Trial_idx_file)['Trial_idx1']
door_open = Trial_idx[:,1]
with h5py.File(filename, "r") as f:
    data = f['estimates']['S_dff'][:]
print(data.shape)
# Rastermap
model = Rastermap(locality=0.2, time_lag_window=20).fit(data)
sorted_order = model.isort

data, door_open_index = generate_continuous_data(data, door_open, [0,len(door_open)-1], [100,200],0.13,1000,30,0.1)
PCA(data, door_open_index, 3)

# Load behavior file
behav_file = path+'behav_score.xlsx'
bf = pd.read_excel(behav_file, header=None)
label = np.squeeze(bf.values)

print(label)
print(data.shape)



fig = plt.figure(figsize=(150,2))
seaborn.heatmap(data[sorted_order],vmin=0,vmax=5,cmap="gray_r")
success_data = []
failure_data = []
for i,e in enumerate(door_open_index):
    if label[i] == "ss" or label[i] =="ms":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='green',linestyle='--', linewidth=0.4)
        success_data.append(data[sorted_order,e-20:e+50])
    elif label[i] == "sf" or label[i] =="mf":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='red',linestyle='--', linewidth=0.4)
        failure_data.append(data[sorted_order,e-20:e+50])
    elif label[i] == "n":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='black',linestyle='--', linewidth=0.4)
    else:
        plt.plot([e-20,e-20],[0,data.shape[0]], color='blue',linestyle='--', linewidth=0.4)
plt.show()

success_data = np.average(success_data,axis=0)
seaborn.heatmap(success_data)
plt.show()
failure_data = np.average(failure_data,axis=0)
seaborn.heatmap(failure_data)
plt.show()

In [ ]:
path = 'G:/M2-4/2025-04-05/'
filename = path+"2025-04-05_cnmf.hdf5"
Trial_idx_file = path+'Trial_idx1.mat'
Trial_idx = scipy.io.loadmat(Trial_idx_file)['Trial_idx1']
door_open = Trial_idx[:,1]
with h5py.File(filename, "r") as f:
    data = f['estimates']['S_dff'][:]
print(data.shape)
# Rastermap
model = Rastermap(locality=0.2, time_lag_window=20).fit(data)
sorted_order = model.isort

data, door_open_index = generate_continuous_data(data, door_open, [0,len(door_open)-1], [100,200],0.13,1000,30,0.1)
PCA(data, door_open_index, 3)

# Load behavior file
behav_file = path+'behav_score.xlsx'
bf = pd.read_excel(behav_file, header=None)
label = np.squeeze(bf.values)

print(label)
print(data.shape)



fig = plt.figure(figsize=(150,2))
seaborn.heatmap(data[sorted_order],vmin=0,vmax=5,cmap="gray_r")
success_data = []
failure_data = []
for i,e in enumerate(door_open_index):
    if label[i] == "ss" or label[i] =="ms":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='green',linestyle='--', linewidth=0.4)
        success_data.append(data[sorted_order,e-20:e+50])
    elif label[i] == "sf" or label[i] =="mf":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='red',linestyle='--', linewidth=0.4)
        failure_data.append(data[sorted_order,e-20:e+50])
    elif label[i] == "n":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='black',linestyle='--', linewidth=0.4)
    else:
        plt.plot([e-20,e-20],[0,data.shape[0]], color='blue',linestyle='--', linewidth=0.4)
plt.show()

success_data = np.average(success_data,axis=0)
seaborn.heatmap(success_data)
plt.show()
failure_data = np.average(failure_data,axis=0)
seaborn.heatmap(failure_data)
plt.show()

In [ ]:
path = 'G:/M2-5/2025-03-27/'
filename = path+"2025-03-27_cnmf.hdf5"
Trial_idx_file = path+'Trial_idx1.mat'
Trial_idx = scipy.io.loadmat(Trial_idx_file)['Trial_idx1']
door_open = Trial_idx[:,1]
with h5py.File(filename, "r") as f:
    data = f['estimates']['S_dff'][:]
print(data.shape)
data, door_open_index = generate_continuous_data(data, door_open, [0,len(door_open)-1], [100,200],0.14,1000,30,0.1)
PCA(data, door_open_index, 3)

In [ ]:
path = 'G:/M2-5/2025-04-06/'
filename = path+"2025-04-06_cnmf.hdf5"
Trial_idx_file = path+'Trial_idx1.mat'
Trial_idx = scipy.io.loadmat(Trial_idx_file)['Trial_idx1']
door_open = Trial_idx[:,1]
with h5py.File(filename, "r") as f:
    data = f['estimates']['S_dff'][:]
print(data.shape)
# Rastermap
model = Rastermap(locality=0.2, time_lag_window=20).fit(data)
sorted_order = model.isort

data, door_open_index = generate_continuous_data(data, door_open, [0,len(door_open)-1], [100,200],0.13,1000,30,0.1)
PCA(data, door_open_index, 3)

# Load behavior file
behav_file = path+'behav_score.xlsx'
bf = pd.read_excel(behav_file, header=None)
label = np.squeeze(bf.values)

print(label)
print(data.shape)



fig = plt.figure(figsize=(150,2))
seaborn.heatmap(data[sorted_order],vmin=0,vmax=5,cmap="gray_r")
success_data = []
failure_data = []
for i,e in enumerate(door_open_index):
    if label[i] == "ss" or label[i] =="ms":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='green',linestyle='--', linewidth=0.4)
        success_data.append(data[sorted_order,e-20:e+50])
    elif label[i] == "sf" or label[i] =="mf":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='red',linestyle='--', linewidth=0.4)
        failure_data.append(data[sorted_order,e-20:e+50])
    elif label[i] == "n":
        plt.plot([e-20,e-20],[0,data.shape[0]], color='black',linestyle='--', linewidth=0.4)
    else:
        plt.plot([e-20,e-20],[0,data.shape[0]], color='blue',linestyle='--', linewidth=0.4)
plt.show()

success_data = np.average(success_data,axis=0)
seaborn.heatmap(success_data)
plt.show()
failure_data = np.average(failure_data,axis=0)
seaborn.heatmap(failure_data)
plt.show()